In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
from src.utils import (
    load_data,
    remove_duplicates_in_coloumn,
    convert_from_degree_to_ciruclar,
    resample_hourly,
    create_time_features,
    find_repeated_indices,
    load_val_dates,
    create_lag_features
)

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [5]:
predictions_1 = pd.read_csv("../data/comparison/model_1_train_predictions.csv")
# predictions_1 = pd.read_csv("../data/comparison/model_2_train_predictions_3.csv")
# predictions_2 = pd.read_csv("../data/comparison/model_2_train_predictions_4.csv")
predictions_2 = pd.read_csv("../data/comparison/model_2_train_predictions_5.csv")
# predictions_2 = pd.read_csv("../data/comparison/model_local.csv")

train_dates = pd.read_csv("../data/processed/train2.csv")

In [6]:
scaler = MinMaxScaler(feature_range=(0, 365))

train_day_of_year = scaler.fit_transform(train_dates[["dayofyear"]])
train_dates["day_of_year"] = train_day_of_year
predictions_1["day_of_year"] = train_day_of_year
predictions_2["day_of_year"] = train_day_of_year
# predictions_local["day_of_year"] = train_day_of_year

In [7]:
val_dates = load_val_dates()
days_of_year = list(set([datetime.strptime(date, "%Y-%m-%d %H:%M:%S").timetuple().tm_yday for date in val_dates]))
winter_days_of_year = [day - 100 for day in days_of_year]

In [9]:
target_predictions_1 = predictions_1[predictions_1["day_of_year"].isin(days_of_year)]["predict"]
target_predictions_2 = predictions_2[predictions_2["day_of_year"].isin(days_of_year)]["pv_measurement"]
# target_predictions_local = predictions_local[predictions_local["day_of_year"].isin(days_of_year)]["predict"]

In [10]:
train_dates[train_dates["day_of_year"].isin(days_of_year)]["pv_measurement"]

4861     0.000000
4862     0.000000
4863     0.001709
4864     0.006837
4865     0.023930
           ...   
74150    0.019416
74151    0.004029
74152    0.000000
74153    0.000000
74154    0.000000
Name: pv_measurement, Length: 4045, dtype: float64

### Model Comparison for Kaggle Days

In [11]:
a = mean_absolute_error(
    target_predictions_1,
    train_dates[train_dates["day_of_year"].isin(days_of_year)]["pv_measurement"],
)
b = mean_absolute_error(
    target_predictions_2,
    train_dates[train_dates["day_of_year"].isin(days_of_year)]["pv_measurement"],
)

print("Improvement?")
print(a - b)

Improvement?
0.11884293237093943


### Model Comparison for All Days

In [13]:
a = mean_absolute_error(
    predictions_1["predict"],
    train_dates["pv_measurement"],
)
b = mean_absolute_error(
    predictions_2["pv_measurement"],
    train_dates["pv_measurement"],
)

print("Improvement?")
print(a - b)

Improvement?
0.09231799802164226


### Model Comparison for Winter Days

In [15]:
a = mean_absolute_error(
    predictions_1[predictions_1["day_of_year"].isin(winter_days_of_year)]["predict"],
    train_dates[train_dates["day_of_year"].isin(winter_days_of_year)]["pv_measurement"],
)
b = mean_absolute_error(
    predictions_2[predictions_2["day_of_year"].isin(winter_days_of_year)]["pv_measurement"],
    train_dates[train_dates["day_of_year"].isin(winter_days_of_year)]["pv_measurement"],
)

print("Improvement?")
print(a - b)

Improvement?
0.06816660627256298
